Baseline code from Notebook: https://www.kaggle.com/rohanrao/automl-tutorial-tps-july-2021

In [ ]:
## define configuration
PATH_TRAIN = '../input/tabular-playground-series-jul-2021/train.csv'
PATH_TEST = '../input/tabular-playground-series-jul-2021/test.csv'

time_limit = 6000

In [ ]:
## prepare data
import gc
import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path

train = pd.read_csv(PATH_TRAIN, index_col="date_time", parse_dates=True)
test = pd.read_csv(PATH_TEST, index_col="date_time", parse_dates=True)

target_carbon_monoxide = train.target_carbon_monoxide.values
target_benzene = train.target_benzene.values
target_nitrogen_oxides = train.target_nitrogen_oxides.values

train['dayoftheweek'] = train.index.dayofweek #Weekly Seasonality
train['hourofday'] = train.index.hour #Daily Seasonality
train['time'] = train.index.astype(np.int64) #Yearly Trend

train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis=1, inplace=True)

test['dayoftheweek'] = test.index.dayofweek #Weekly Seasonality
test['hourofday'] = test.index.hour #Daily Seasonality
test['time'] = test.index.astype(np.int64) #Yearly Trend

In [ ]:
## installing packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q graphviz --upgrade

In [ ]:
## imports
from autogluon.tabular import TabularPredictor, TabularDataset

In [ ]:
## run model for carbon monoxide
train['target'] = target_carbon_monoxide

autogluon_carbon_monoxide = TabularPredictor(
    label='target'
)

autogluon_carbon_monoxide.fit(train_data=train, 
                              time_limit=time_limit, 
                              presets='best_quality',
                              num_stack_levels = 3,
                              num_bag_folds = 5,
                              num_bag_sets = 1,)

del train['target']

In [ ]:
## check leaderboard for carbon monoxide
autogluon_carbon_monoxide.leaderboard()

In [ ]:
## run model for benzene
train['target'] = target_benzene

autogluon_benzene = TabularPredictor(
    label='target'
)

autogluon_benzene.fit(train_data=train, 
                      time_limit=time_limit, 
                      presets='best_quality',
                      num_stack_levels = 3,
                      num_bag_folds = 5,
                      num_bag_sets = 1,)

del train['target']

In [ ]:
## check leaderboard for benzene
autogluon_benzene.leaderboard()

In [ ]:
## run model for nitrogen oxides
train['target'] = target_nitrogen_oxides

autogluon_nitrogen_oxides = TabularPredictor(
    label='target'
)

autogluon_nitrogen_oxides.fit(train_data=train, 
                              time_limit=time_limit, 
                              presets='best_quality',
                              num_stack_levels = 3,
                              num_bag_folds = 5,
                              num_bag_sets = 1,)

del train['target']

In [ ]:
## check leaderboard for nitrogen oxides
autogluon_nitrogen_oxides.leaderboard()

In [ ]:
test.columns

In [ ]:
## generate predictions
preds_autogluon_carbon_monoxide = autogluon_carbon_monoxide.predict(TabularDataset(test))
preds_autogluon_benzene = autogluon_benzene.predict(TabularDataset(test))
preds_autogluon_nitrogen_oxides = autogluon_nitrogen_oxides.predict(TabularDataset(test))

In [ ]:
dt = pd.DataFrame(test.index.values,  columns=['date_time'])

## create submission
submission = pd.DataFrame({
    'date_time': preds_autogluon_carbon_monoxide.index,
    'target_carbon_monoxide': preds_autogluon_carbon_monoxide,
    'target_benzene': preds_autogluon_benzene,
    'target_nitrogen_oxides': preds_autogluon_nitrogen_oxides
})

submission.head()

In [ ]:
## save submission
PATH_AUTOGLUON_SUBMISSION = 'submission_autogluon_' + str(time_limit) + '.csv'

PATH_AUTOGLUON_SUBMISSION

In [ ]:
submission.to_csv(PATH_AUTOGLUON_SUBMISSION, index=False)

In [ ]:
## clear memory
shutil.rmtree('AutogluonModels')

gc.collect()